### Calculation of the ratio S/N

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
#In[2]:
# define function
import src.SAT_function as data_process
import src.Data_Preprocess as preprosess

In [ ]:
import src.slurm_cluster as scluster
client, scluster = scluster.init_dask_slurm_cluster()

### Input both the forced and ICV_std trend data 

In [ ]:
# dir_in= '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/LE_evaluation/Fig3_CanESM5/output/'

# CanESM5_forced_ds = xr.open_mfdataset(dir_in + 'CanESM5_forced_segmented_trend.nc')
# print(CanESM5_forced_ds)

In [ ]:
dir_ICV = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/LE_evaluation/Fig3_CanESM5/output/'
# Input multiple runs ICV into one dataset with new variable dimension 'run'
variable_indices = np.arange(0, 50, 1).astype(str)
segment_lengths = np.arange(10, 74, 1).astype(str)

# Directory containing input files
dir_ICV = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/LE_evaluation/Fig3_CanESM5/output/'

# Initialize an empty list to collect datasets
datasets = []

for var in variable_indices:
    print(f"Processing run {var}")
    file = dir_ICV + f'CanESM5_ICV_segments_1850_2022_run{var}_trend.nc'
    
    # Open the dataset for the current run
    try:
        ds = xr.open_mfdataset(file, engine="netcdf4",combine='by_coords')
    except FileNotFoundError:
        print(f"File not found: {file}")
        continue
    
    # Expand dimensions to add a `run` dimension
    ds = ds.expand_dims(run=[int(var)])
    datasets.append(ds)

# Combine all datasets along the `run` dimension
if datasets:
    CanESM5_ICV_ds = xr.concat(datasets, dim="run")
    # change the run dimension range from 0-49 to 1-50
    CanESM5_ICV_ds['run'] = np.arange(1, 51, 1)
    print("Datasets successfully combined!")
else:
    print("No datasets were combined.")

In [ ]:
CanESM5_ICV_ds

In [ ]:
# # Save the combined dataset to a NetCDF file
# output_file = dir_ICV + 'CanESM5_ICV_combined_1850_2022.nc'
# CanESM5_ICV_ds.to_netcdf(output_file)
# print(f"Combined dataset saved to: {output_file}")

In [ ]:
# calculate the standard deviation of the ICV
# define function to calculate the standard deviation of the trend pattern of each interval of segments
def std_trend_pattern(data):
    """
    data: 3D array with dimensions [segment, lat, lon]
    segment_number: number of segments
    """
    
    # Calculate the standard deviation of the trend pattern of each interval of segments
    std_trend_pattern = np.nanstd(data, axis=0)
    
    return std_trend_pattern

In [ ]:
# calculate standard deviation of the trend pattern of each interval of segments
# apply_ufunc to apply the function to the dataset along the 'segment' dimension
CanESM5_ICV_noise_ds = xr.Dataset()
for segment in segment_lengths:
    print(f"Processing segment {segment}")
    CanESM5_ICV_noise_ds[f'std_trend_{segment}'] = xr.apply_ufunc(std_trend_pattern, 
        CanESM5_ICV_ds[f'trend_{segment}'].chunk({'run': -1}), 
        input_core_dims=[['segment', 'lat', 'lon']], 
        output_core_dims=[['lat', 'lon']], 
        vectorize=True, dask='parallelized', 
        output_dtypes=[float])

In [ ]:
# save the standard deviation of the trend pattern of each interval of segments to a NetCDF file
output_file = dir_ICV + 'CanESM5_ICV_noise_std_trend_pattern_1850_2022.nc'
CanESM5_ICV_noise_ds.to_netcdf(output_file)
print(f"Standard deviation of the trend pattern of each interval of segments saved to: {output_file}")

In [ ]:
CanESM5_ICV_noise_ds

### 